# Analysis of UQ runs - DEMO 2016 initial uncertainties VINE


## Results summary

|   | aspect | tbrnmn   | etath   | ralpne   |
|---|--------|----------|---------|----------|
| Design point | 3.32   | 17542.19 | 0.37    | 0.06857  |
| Optimised point | 2.83   | 15293.11 | 0.39    | 0.07316  |
| Delta | -0.49  | -2249.07 | 0.01863 | 0.00459 |

## Import the Uncertainty Data and create object

## Calculate Sensitivity Indices and Reliability

In [ ]:
%load_ext autoreload
%autoreload 1
from pathlib import Path
from bokeh.plotting import figure, show
import sys
import pandas as pd
sys.path.append('/home/graeme/process_uq/')
sys.path.append('/home/graeme/PROCESS_SALib/')
from uq_analysis import UncertaintyData, Copula, ConfidenceAnalysis
# from bokeh.plotting import figure 
from bokeh.io import output_notebook
output_notebook()
proj_dir = str(Path.cwd())+"/"
proj_dir = "/home/graeme/easyVVUQ-process/demo_analysis/"
figure_of_merit = "rmajor"
input_names = [
            "coreradius",
            "ralpne",
            "psepbqarmax",
            "tbrnmn",
            "etaech",
            "pinjalw",
            "triang",
            "alstroh",
            "sig_tf_case_max",
            "walalw",
            "sig_tf_wp_max",
            "aspect",
            "etath"
        ]
uq_data = UncertaintyData(proj_dir, figure_of_merit, input_parameters = input_names, use_scoping_data=False)



In [ ]:
uq_data.calculate_sensitivity(figure_of_merit)
uq_data.calculate_failure_probability()
print("Number of samples: ", len(uq_data.uncertainties_df))
print("Number of converged runs: ",uq_data.number_of_converged_runs,)
print("Failure Rate: ", uq_data.failure_probability,"+/-", uq_data.failure_cov)


In [ ]:
itv=[
            "bt",
            "te",
            "beta",
            "dene",
            "tfcth",
            "wallmw",
            "ohcth",
            "bigq",
            "bore",
            "betalim",
            "coheof",
            "cohbop",
            "fvsbrnni",
            # "itvar019",
            # "itvar020",
            "jwptf",
            # "vtfskv",
            # "vdalw",
            "tdmptf",
            "thkcas",
            # "thwcndut",
            "fcutfsu",
             "cpttf",
            "plhthresh",
            "tmargtf",
            "tmargoh",
            "oh_steel_frac",
            "pdivt",
            "powfmw",
            "rmajor",]

## Regional Sensitivity - what causes PROCESS to converge?

In [ ]:
uq_data.convergence_regional_sensitivity_analysis(uq_data.input_names)
uq_data.plot_sumsq_sensitivity()
significant_conv_vars = uq_data.find_significant_parameters(uq_data.sumsq_sensitivity_df,"unconverged",0.0).tolist()
print(significant_conv_vars)

"aspect" = aspect ratio

"tbrnmn" = minimum required burn time

"etath" = electric to thermal conversion efficiency

"ralpne" = thermal alpha/electron density.

## Find the iteration variables which are influential for the significant convergence variables

In [ ]:
copula = Copula(uq_data.converged_df,significant_conv_vars+["powfmw","rmajor"],copula_type="unbounded")
copula.calculate_copula(synthetic_sample_size=200)
copula.copula_data_dict()
print(copula.copula_dict)
copula.plot_correlation_matrix(copula.copula_dict["correlation"])
copula.copula.tau_mat
correlation_df = pd.DataFrame(copula.copula.tau_mat,columns=significant_conv_vars+["rmajor"],index=significant_conv_vars+["rmajor"])

In [ ]:
model_parameters = significant_conv_vars
vcopula = Copula(uq_data.converged_df,model_parameters,copula_type="vine")
vcopula.calculate_copula(synthetic_sample_size=500)
vcopula.copula.tau_mat
correlation_df = pd.DataFrame(vcopula.copula.tau_mat,columns=model_parameters,index=model_parameters)

In [ ]:
vcopula.plot_correlation_matrix(correlation_df)

In [ ]:
custom_data = {"coreradius":0.8}#,"tbrnmn":10000,"ralpne":0.06}
vconv_copula_analysis=ConfidenceAnalysis(uq_data,vcopula, weight_confidence=1.0,weight_overlap=0.1, num_intervals=10)#,custom_data_point=custom_data)
show(vconv_copula_analysis.create_graph_grid(significant_conv_vars))
show(vconv_copula_analysis.create_datatable(significant_conv_vars))

In [ ]:
vconv_copula_analysis.plot_network(vconv_copula_analysis.correlation_network(correlation_df,threshold=0.5))